In [20]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report
import optuna
import pandas as pd

In [21]:
df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [22]:
X = df.drop(columns=['Class'])
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [30]:
def objective(trial):

    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'use_label_encoder': False,
        'n_estimators': trial.suggest_int('n_estimators', 200, 700),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.05),
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 2, 10),
        'scale_pos_weight': len(y_train[y_train == 0]) / len(y_train[y_train == 1])
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []

    for train_index, val_index in kf.split(X_train_scaled, y_train):
        X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model = xgb.XGBClassifier(**param)
        model.fit(X_train_fold, y_train_fold)

        y_pred_proba = model.predict_proba(X_val_fold)[:, 1]

        auc = roc_auc_score(y_val_fold, y_pred_proba)
        auc_scores.append(auc)

    mean_auc = sum(auc_scores) / len(auc_scores)
    return mean_auc

In [28]:
study = optuna.create_study(direction='maximize', study_name='Fraud_detection')
study.optimize(objective, n_trials=50, timeout=1200)

best_params = study.best_params
print(f"Best hparams: {best_params}")

[I 2024-08-09 16:58:12,279] A new study created in memory with name: Fraud_detection
[I 2024-08-09 16:58:50,632] Trial 0 finished with value: 0.982369086493183 and parameters: {'n_estimators': 696, 'learning_rate': 0.022952455701782462, 'max_depth': 4, 'subsample': 0.9031913805230565, 'colsample_bytree': 0.9403564569789717, 'gamma': 9.182687874121811}. Best is trial 0 with value: 0.982369086493183.
[I 2024-08-09 16:59:46,685] Trial 1 finished with value: 0.9806832896177118 and parameters: {'n_estimators': 574, 'learning_rate': 0.03424899673125994, 'max_depth': 11, 'subsample': 0.5757405506806595, 'colsample_bytree': 0.8507368152449812, 'gamma': 2.5127208216181707}. Best is trial 0 with value: 0.982369086493183.
[I 2024-08-09 17:00:27,146] Trial 2 finished with value: 0.9833172884064239 and parameters: {'n_estimators': 555, 'learning_rate': 0.0028722410497979863, 'max_depth': 6, 'subsample': 0.8901232833682166, 'colsample_bytree': 0.7295859166981585, 'gamma': 3.1449839329516855}. Best i

Best hparams: {'n_estimators': 463, 'learning_rate': 0.03173132760152346, 'max_depth': 10, 'subsample': 0.9451586859466636, 'colsample_bytree': 0.5477628758346557, 'gamma': 3.1188250719555723}


In [29]:
final_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='auc', scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]))
final_model.fit(X_train_scaled, y_train)

y_pred_proba = final_model.predict_proba(X_test_scaled)[:, 1]
y_pred = final_model.predict(X_test_scaled)

auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test Dataset AUC: {auc:.4f}")

print(classification_report(y_test, y_pred))

Test Dataset AUC: 0.9641
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.87      0.84      0.85        49

    accuracy                           1.00     28481
   macro avg       0.94      0.92      0.93     28481
weighted avg       1.00      1.00      1.00     28481

